In [1]:
# this notebook iterate over neuron 4D files (already masked and filtered) 
# and stream-pipeline the defined analysis steps 
# and save the output into one dataframe per neuron with the following columns
# output_cols: timepoints, vol_all, vol_stable, growthR_all, growthR_stable, transient_px_N, transient_px_per ....

In [2]:
data_path = '/Users/aichah/Desktop/TLI/data/test_data/'
save_path = '/Users/aichah/Desktop/TLI/data/test_data/output/'
entry_points = [[0, 27, 234, 181],
                [0, 26, 255, 203],
                [0, 28, 172, 193],
                [0, 36, 178, 184],
                [0, 22, 216, 105],
                [0, 45, 232, 194]] # define the entry point for each neuron in the folder
stab_limit = 5 #(No. of timepoints for stable branches calculation)

if save_path[-1] != '/':
    save_path += '/'

In [3]:
import os
import utils.analysis as analysis
import utils.datautils as datautils
import matplotlib.pyplot as plt
import pandas as pd
import tifffile as tif
import numpy as np
import re
from tqdm import tqdm
from timeit import default_timer as timer

In [4]:
files_list = datautils.get_file_names(data_path, group_by='', 
                                        order=True, nested_files=False, 
                                        criteria='tif')

first 5 files
['/Users/aichah/Desktop/TLI/data/test_data/220209_P36_N2_mask_flipV_JESUS.tif', '/Users/aichah/Desktop/TLI/data/test_data/220209_P36_N1_mask_flipV_JESUS_new_mask.tif', '/Users/aichah/Desktop/TLI/data/test_data/211121_P36.5_N3_GFP_JESUS_label.tif', '/Users/aichah/Desktop/TLI/data/test_data/211113_P36_N2_GFP_JESUS_new_mask.tif', '/Users/aichah/Desktop/TLI/data/test_data/211113_P36_N1_mask3_JESUS.tif']


In [5]:
for i, file in tqdm(enumerate(files_list)):
    file_name = os.path.basename(file)
    start_age = re.search('P(.*)_N', file_name)
    start_age = float(start_age.group(1))
    neuron = tif.imread(file)
    neuron[neuron != 0] = 1
    lifetimes = analysis.cal_lifetimes(neuron, save=False, 
                                        save_path=save_path, save_file=file_name, 
                                        xy_pixel=0.076, z_pixel=0.4)
    stable_neuron = analysis.stable_N(lifetimes, stab_limit=stab_limit, save=False, 
                                        save_path=save_path, save_file=file_name, 
                                        xy_pixel=0.076, z_pixel=0.4)
    timepoints = [start_age+(i*0.25) for i in np.arange(len(lifetimes))]   
    output_results = pd.DataFrame({'timepoints':timepoints})
    output_results = analysis.N_volume(neuron, stable_neuron, normalize=False, 
                                    start_t=start_age, plot=False, 
                                    save=False, save_path=save_path, 
                                    save_file='Vol_'+file_name)
    vol_all = lifetimes.copy()
    vol_all[vol_all > 0] = 1
    output_results['vol_all1'] = vol_all.sum(axis=(1,2,3))
    output_results['vol_stable1'] = stable_neuron.sum(axis=(1,2,3))     
    output_results['growthR_all'] = output_results.vol_all.diff()/output_results.vol_all
    output_results['growthR_stable'] = output_results.vol_stable.diff()/output_results.vol_stable
    output_results['trans_per'] = (output_results.vol_all - output_results.vol_stable)/output_results.vol_stable
    DGIs = analysis.calculate_DGI(entry_points[i], neuron, 
                                  start_t=start_age, 
                                  save=False, save_path='', 
                                  save_file='')
    output_results = pd.merge(output_results, DGIs, on='timepoints')
    output_file = save_path+file_name+'_output.csv'
    output_file = output_file.replace('.tif','')
    output_results.to_csv(output_file, sep=';')

100%|██████████| 91/91 [00:04<00:00, 18.72it/s]
6it [03:17, 32.99s/it]


IndexError: list index out of range

In [5]:
neuron = tif.imread(files_list[0])
neuron[neuron != 0] = 1
lifetimes = analysis.cal_lifetimes(neuron, save=False, 
                                    save_path=save_path, save_file='', 
                                    xy_pixel=0.076, z_pixel=0.4)
# print(i, 'step1', round(timer(),3)-start)
stable_neuron = analysis.stable_N(lifetimes, stab_limit=stab_limit, save=False, 
                                    save_path=save_path, save_file='', 
                                    xy_pixel=0.076, z_pixel=0.4)

100%|██████████| 87/87 [00:04<00:00, 17.92it/s]


In [28]:
timepoints = [36+(i*0.25) for i in np.arange(len(lifetimes))]
output_results = pd.DataFrame({'timepoints':timepoints})
# results['lifetimes'] = lifetimes.sum(axis=(1,2,3))
vol_all = lifetimes.copy()
vol_all[vol_all > 0] = 1
output_results['vol_all'] = vol_all.sum(axis=(1,2,3))
output_results['vol_stable'] = stable_neuron.sum(axis=(1,2,3))
output_results['growthR_all'] = output_results.vol_all.diff()/output_results.vol_all
output_results['growthR_stable'] = output_results.vol_stable.diff()/output_results.vol_stable
output_results['trans_per'] = (output_results.vol_all - output_results.vol_stable)/output_results.vol_stable
output_results['trans_per'] = round(output_results['trans_per'],4)

In [29]:
output_file = save_path+'test'+'_output.csv'
output_file = output_file.replace('.tif','')
output_results.to_csv(output_file, sep=';')

In [11]:
print(lifetimes.shape, lifetimes.max())
lifetimes.sum(axis=(1,2,3))

(88, 43, 400, 400) 88.0


array([ 123443.,  239413.,  323224.,  385346.,  487428.,  484679.,
        558009.,  635354.,  691009.,  758363.,  827638.,  920673.,
        966155., 1036822., 1101022., 1178023., 1270735., 1282131.,
       1367939., 1449523., 1515779., 1547416., 1611608., 1654075.,
       1669984., 1744900., 1839306., 1808726., 1899697., 1950807.,
       2014772., 2084854., 2136871., 2209047., 2320012., 2412037.,
       2502678., 2591585., 2686672., 2765242., 2842314., 2877460.,
       2970268., 3077839., 3108871., 3222517., 3336783., 3444485.,
       3559534., 3652819., 3770924., 3866230., 3933022., 4038608.,
       4139690., 4227536., 4305086., 4409369., 4505620., 4564247.,
       4662106., 4773230., 4890997., 4963604., 5040907., 5140954.,
       5236425., 5319872., 5428655., 5525791., 5607824., 5696629.,
       5764142., 5873177., 5952445., 6058495., 6157246., 6218669.,
       6312947., 6320229., 6361238., 6479895., 6510165., 6628513.,
       6720061., 6772669., 6879097., 6946732.])

In [5]:
for i, file in tqdm(enumerate(files_list)):
    file_name = os.path.basename(file)
    start_age = re.search('P(.*)_N', file_name)
    start_age = float(start_age.group(1))
    neuron = tif.imread(file)
    neuron[neuron != 0] = 1
    lifetimes = analysis.cal_lifetimes(neuron, save=False, 
                                        save_path=save_path, save_file=file_name, 
                                        xy_pixel=0.076, z_pixel=0.4)
    stable_neuron = analysis.stable_N(lifetimes, stab_limit=stab_limit, save=False, 
                                        save_path=save_path, save_file=file_name, 
                                        xy_pixel=0.076, z_pixel=0.4)
    timepoints = [start_age+(i*0.25) for i in np.arange(len(lifetimes))]     
    output_results = analysis.N_volume(neuron, stable_neuron, normalize=False, 
                                        start_t=start_age, plot=False, 
                                        save=False, save_path=save_path, 
                                        save_file='Vol_'+file_name)
    output_results['growthR_all'] = output_results.vol_all.diff()/output_results.vol_all
    output_results['growthR_stable'] = output_results.vol_stable.diff()/output_results.vol_stable
    output_results['trans_per'] = (output_results.vol_all - output_results.vol_stable)/output_results.vol_stable
    DGIs = analysis.calculate_DGI(entry_points[i], neuron, 
                                  start_t=start_age, 
                                  save=False, save_path='', 
                                  save_file='')
    output_results = pd.merge(output_results, DGIs, on='timepoints')
    output_file = save_path+file_name+'_output.csv'
    output_file = output_file.replace('.tif','')
    output_results.to_csv(output_file, sep=';')
    

100%|██████████| 91/91 [00:04<00:00, 18.57it/s]
6it [02:48, 28.00s/it]


IndexError: list index out of range

In [7]:
neuron = tif.imread(files_list[0])
neuron[neuron != 0] = 1
lifetimes = analysis.cal_lifetimes(neuron, save=False, 
                                    save_path=save_path, save_file='', 
                                    xy_pixel=0.076, z_pixel=0.4)
# print(i, 'step1', round(timer(),3)-start)
stable_neuron = analysis.stable_N(lifetimes, stab_limit=stab_limit, save=False, 
                                    save_path=save_path, save_file='', 
                                    xy_pixel=0.076, z_pixel=0.4)

100%|██████████| 87/87 [00:04<00:00, 18.10it/s]


In [9]:
stable_neuron.max(), lifetimes.max(), lifetimes.shape, stable_neuron.shape

(1.0, 88.0, (88, 43, 400, 400), (88, 43, 400, 400))

In [17]:
len(stable_neuron>0), stable_neuron.sum()

(88, 10364140.0)

In [18]:
tt = lifetimes[lifetimes>0]
tt.sum()

295148933.0

In [26]:
lifetimes.sum(axis=(1,2,3))

array([ 123443.,  239413.,  323224.,  385346.,  487428.,  484679.,
        558009.,  635354.,  691009.,  758363.,  827638.,  920673.,
        966155., 1036822., 1101022., 1178023., 1270735., 1282131.,
       1367939., 1449523., 1515779., 1547416., 1611608., 1654075.,
       1669984., 1744900., 1839306., 1808726., 1899697., 1950807.,
       2014772., 2084854., 2136871., 2209047., 2320012., 2412037.,
       2502678., 2591585., 2686672., 2765242., 2842314., 2877460.,
       2970268., 3077839., 3108871., 3222517., 3336783., 3444485.,
       3559534., 3652819., 3770924., 3866230., 3933022., 4038608.,
       4139690., 4227536., 4305086., 4409369., 4505620., 4564247.,
       4662106., 4773230., 4890997., 4963604., 5040907., 5140954.,
       5236425., 5319872., 5428655., 5525791., 5607824., 5696629.,
       5764142., 5873177., 5952445., 6058495., 6157246., 6218669.,
       6312947., 6320229., 6361238., 6479895., 6510165., 6628513.,
       6720061., 6772669., 6879097., 6946732.])

In [6]:
for i, file in tqdm(enumerate(files_list)):
    # start = round(timer(),3)
    file_name = os.path.basename(file)
    start_age = re.search('P(.*)_N', file_name)
    start_age = float(start_age.group(1))
    neuron = tif.imread(file)
    neuron[neuron != 0] = 1
    lifetimes = analysis.cal_lifetimes(neuron, save=False, 
                                        save_path=save_path, save_file=file_name, 
                                        xy_pixel=0.076, z_pixel=0.4)
    # print(i, 'step1', round(timer(),3)-start)
    stable_neuron = analysis.stable_N(lifetimes, stab_limit=stab_limit, save=False, 
                                        save_path=save_path, save_file=file_name, 
                                        xy_pixel=0.076, z_pixel=0.4)
    timepoints = [start_age+(i*0.25) for i in np.arange(len(lifetimes))]     
    # print(i, 'step2', round(timer(),3)-start)
    # output_results = analysis.N_volume(neuron, stable_neuron, normalize=False, 
    #                                     start_t=start_age, plot=False, 
    #                                     save=False, save_path=save_path, 
    #                                     save_file='Vol_'+file_name)
    output_results = pd.DataFrame({'timepoints':timepoints})
    output_results['vol_all'] = len(np.where(lifetimes>0)[0])
    output_results['vol_stable'] = len(np.where(stable_neuron>0)[0])
    # output_results['vol_stable2'] = len(np.where(lifetimes>=stab_limit)[0])
    # print(i, 'step3', round(timer(),3)-start)
    output_results['growthR_all'] = output_results.vol_all.diff()/output_results.vol_all
    output_results['growthR_stable'] = output_results.vol_stable.diff()/output_results.vol_stable
    output_results['trans_per'] = (output_results.vol_all - output_results.vol_stable)/output_results.vol_stable
    # output_results.growthR_all = round(output_results.growthR_all, 3)
    # output_results.growthR_stable = round(output_results.growthR_stable, 3)
    # output_results['trans_per'] = round(output_results['trans_per'], 3)
    # print(i, 'step4', round(timer(),3)-start)
    # output_transient = analysis.trans_px_levels(lifetimes, stab_limit=stab_limit, 
    #                                             start_t=start_age, plot=False, 
    #                                             save=False, save_path='', 
    #                                             save_file='')
    # output_results = pd.merge(output_results, output_transient, on='timepoints')
    # # print(i, 'step5', round(timer(),3)-start)
    # output_transient = analysis.trans_px_levels2(neuron, stable_neuron, 
    #                                             start_t=start_age, plot=False, 
    #                                             save=False, save_path='', 
    #                                             save_file='')
    # output_results = pd.merge(output_results, output_transient, on='timepoints')
    # print(i, 'step6', round(timer(),3)-start)
    DGIs = analysis.calculate_DGI(entry_points[i], neuron, 
                                  start_t=start_age, 
                                  save=False, save_path='', 
                                  save_file='')
    output_results = pd.merge(output_results, DGIs, on='timepoints')
    # print(i, 'step7', round(timer(),3)-start)
    output_file = save_path+file_name+'_output.csv'
    output_file = output_file.replace('.tif','')
    output_results.to_csv(output_file, sep=';')

    # del neuron, stable_neuron, lifetimes, DGIs, output_results, output_transient

100%|██████████| 91/91 [00:06<00:00, 13.63it/s]


In [1]:
neuron

NameError: name 'neuron' is not defined